<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/Cadec_twadr_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,24,heart beat for being too strong
2,62,feeling proper psycho
3,94,anxiety
4,133,night terrors


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-2.test.csv",header=None)
test.head()

,0,1
0,13,It's 6 am and this adderall got me reevaluatin...
1,24,racing heart
2,56,persistent painful erection
3,57,hypertrophic scarring
4,94,anxiety


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-cadec.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'the',
 'and',
 'to',
 'my',
 'pain',
 'in',
 'of',
 'a',
 'it',
 'me',
 'mood',
 'increased',
 'for',
 '-',
 'have',
 'was',
 'drug',
 'lipitor',
 '(',
 ')',
 'this',
 'disorder',
 'on',
 'withdrawal',
 'abnormal',
 'not',
 'symptoms',
 'had',
 'anxiety',
 'that',
 'is',
 'with',
 'weight',
 'muscle',
 'like',
 'but',
 'taking',
 'feeling',
 'decreased',
 "n't",
 'disorders',
 'after',
 'up',
 'sleeplessness',
 'syndrome',
 'behavior',
 'so',
 'sleep',
 'blood',
 'got',
 'insomnia',
 'diseases',
 'all',
 'feel',
 'energy',
 'disturbance',
 'from',
 'finding',
 'am',
 'depression',
 'side',
 'back',
 'somnolence',
 '!',
 'loss',
 'cholesterol',
 'day',
 'been',
 'heart',
 'be',
 'effects',
 'mental',
 'skin',
 'gain',
 'as',
 'appetite',
 'take',
 'body',
 '/',
 'at',
 'nausea',
 'acute',
 'about',
 'no',
 'off',
 'mg',
 'now',
 'fatigue',
 'years',
 'do',
 'has',
 'dizziness',


In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.284174,3.469149,0.426479,08:06


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.290010,3.041438,0.484319,08:02


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.044009,3.007803,0.487751,08:02


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.017215,2.979372,0.490792,08:01


In [25]:
learn.save_encoder('TwADR-cadec_fold2_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-cadec_fold2_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.191018,6.355796,0.107143,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.243910,5.409770,0.164286,00:23


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.808930,5.023649,0.221429,00:58


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.467628,4.650749,0.235714,00:58
1,5.283749,4.490212,0.264286,00:58


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.982731,4.154720,0.300000,01:00
1,4.797934,4.036478,0.278571,00:59


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.572432,3.841286,0.314286,00:56
1,4.429288,3.805712,0.328571,00:58


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.203556,3.661320,0.321429,00:57
1,4.056888,3.602844,0.357143,00:58


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.911967,3.535796,0.364286,00:56
1,3.721219,3.467612,0.378571,00:57


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.555582,3.448079,0.392857,00:56
1,3.486620,3.388064,0.335714,01:00
2,3.301610,3.291774,0.378571,00:59
3,3.135826,3.285598,0.407143,00:56


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.002877,3.303468,0.364286,01:00
1,2.908903,3.219060,0.421429,01:03
2,2.742855,3.200720,0.407143,00:58
3,2.578526,3.186763,0.421429,00:57


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.438857,3.200184,0.421429,00:59
1,2.373379,3.286238,0.392857,00:58
2,2.206403,3.248786,0.400000,01:02
3,2.069679,3.234032,0.421429,01:02


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.910596,3.247226,0.414286,00:56
1,1.899622,3.354410,0.435714,00:58
2,1.769787,3.343515,0.400000,01:00
3,1.630823,3.351908,0.414286,00:57


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.573858,3.373995,0.435714,01:00
1,1.558301,3.458032,0.414286,01:01
2,1.409669,3.450265,0.400000,00:57
3,1.305611,3.464326,0.400000,00:58


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.181529,3.508908,0.357143,00:58
1,1.169187,3.532417,0.400000,01:02
2,1.110904,3.555407,0.421429,01:01
3,1.023707,3.604430,0.421429,00:57


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.938433,3.544840,0.407143,00:59
1,0.965595,3.666746,0.421429,00:59
2,0.889679,3.710457,0.414286,01:01
3,0.811069,3.730391,0.414286,01:01


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.782234,3.729173,0.421429,00:58
1,0.757459,3.774719,0.421429,00:59


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.718606,3.891943,0.400000,00:57
1,0.726332,3.940521,0.435714,00:59


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.649350,3.892069,0.407143,00:59


In [44]:
# save the best model

learn.save_encoder('TwADR-cadec_fold2')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.676654,4.107844,0.414286,00:58


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.589719,4.068735,0.407143,01:04


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.591183,4.034779,0.421429,00:59


In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.581688,4.083788,0.392857,01:04


In [52]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.593828,4.155576,0.392857,01:02


# Part three: Predict on the test dataset

In [53]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [54]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1196, 2104, 56, 792, 978, 250, 2048, 2048, 189, 2048, 217, 1521, 233, 1936, 233, 1647, 301, 1787, 1787, 1787, 1813, 1529, 445, 1230, 582, 908, 792, 720, 744, 744, 749, 749, 749, 887, 1349, 2032, 792, 792, 2125, 418, 824, 1295, 265, 1184, 1356, 974, 974, 974, 978, 1507, 1019, 359, 359, 1767, 620, 1099, 2032, 2194, 1099, 1136, 1142, 1184, 1184, 1427, 2147, 1265, 1265, 1295, 1295, 852, 1295, 1308, 1373, 1340, 1340, 1507, 1973, 2191, 1099, 1356, 2194, 582, 978, 1417, 1701, 1526, 1529, 1062, 1099, 301, 109, 1597, 1597, 1196, 2032, 1597, 1099, 1647, 1073, 1526, 300, 1767, 1718, 1767, 1718, 1718, 1718, 1718, 1718, 1099, 1768, 1099, 1787, 1787, 1787, 1787, 393, 1196, 418, 1810, 2191, 1073, 233, 1936, 109, 233, 1936, 1936, 1099, 1099, 1356, 923, 1767, 579, 1942, 418, 1099, 1529, 233, 1099, 2023, 2191, 393]
[13, 24, 56, 57, 94, 113, 189, 189, 189, 189, 217, 224, 233, 233, 233, 265, 301, 393, 393, 393, 396, 418, 445, 452, 582, 605, 609, 720, 744, 745, 749, 749, 749, 749, 755, 791, 792, 7

In [55]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

56
0.3916083916083916
